In [1]:
!pip install transformers peft accelerate \
    -U --no-index --find-links /kaggle/input/lmsys-wheel-files

Looking in links: /kaggle/input/lmsys-wheel-files
Processing /kaggle/input/lmsys-wheel-files/peft-0.11.1-py3-none-any.whl


In [2]:
!md5sum /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/*

bde11b53dd39402bbac347e1677d5b59  /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/adapter.bin
b9a7a9cfe915c587f6fa31268b8529b0  /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/added_tokens.json
72dacf1de43bc354dc3c521e44ed0b24  /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/merges.txt
5708ee87a0925280424b5caa0906752f  /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/special_tokens_map.json
44bfe8e8fafd3406aba831ccfca45f01  /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/tokenizer.json
11fa2306e41c60e3faf955214555d7cf  /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/tokenizer_config.json
613b8e4a622c4a2c90e9e1245fc540d6  /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/vocab.json


In [3]:
%%capture
!pip install --no-index /kaggle/input/bitsandbytes0-42-0/bitsandbytes-0.42.0-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/optimum-1.21.2-py3-none-any.whl --find-links=/kaggle/input/bitsandbytes0-42-0
!pip install --no-index  /kaggle/input/bitsandbytes0-42-0/auto_gptq-0.7.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl --find-links=/kaggle/input/bitsandbytes0-42-0

In [4]:
# !torchrun --standalone

In [5]:
# !pip list

In [6]:
%%writefile main.py

import pickle as pkl
import sys
BASE_MODEL = sys.argv[1]
BEST_WEIGHT = sys.argv[2]

SAVE_FILE_NAME = sys.argv[3]

VALID = int(sys.argv[4])

print(f"BASE_MODEL = {BASE_MODEL}")
print(f"BEST_WEIGHT = {BEST_WEIGHT}")
print(f"SAVE_FILE_NAME = {SAVE_FILE_NAME}")
print(f"VALID = {VALID}")


import torch
import torch.distributed as dist  
import os
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '5678'

dist.init_process_group(backend='nccl')
# dist.init_process_group(backend='gloo')


local_rank, world_size = dist.get_rank(), dist.get_world_size()
print(f"local_rank: {local_rank}, world_size: {world_size}")

# 设置 device 很重要！！要不然后面的 inputs.to(device) 就不知道 device 是哪个了！
device = torch.device(f"cuda:{local_rank}" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")



# from tqdm.auto import tqdm
from tqdm import tqdm
from bs4 import BeautifulSoup
import gc
import pandas as pd
import pickle
import sys
import numpy as np
# from tqdm.autonotebook import trange
from sklearn.model_selection import GroupKFold
import json
import torch
from numpy.linalg import norm
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel,BitsAndBytesConfig
from peft import (
    LoraConfig,
    get_peft_model,
)
import json
import copy
import warnings
# warnings.filterwarnings('ignore')


def apk(actual, predicted, k=25):
    """
    Computes the average precision at k.
    
    This function computes the average prescision at k between two lists of
    items.
    
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    
    if not actual:
        return 0.0

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        # first condition checks whether it is valid prediction
        # second condition checks if prediction is not repeated
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    return score / min(len(actual), k)

def mapk(actual, predicted, k=25):
    """
    Computes the mean average precision at k.
    
    This function computes the mean average prescision at k between two lists
    of lists of items.
    
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
        
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

def batch_to_device(batch, target_device):
    """
    send a pytorch batch to a device (CPU/GPU)
    """
    for key in batch:
        if isinstance(batch[key], Tensor):
            batch[key] = batch[key].to(target_device)
    return batch

def last_token_pool(last_hidden_states: Tensor,
                    attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'


from torch.nn.parallel import DataParallel 
from torch.utils.data import DataLoader, Dataset, DistributedSampler  

class DatasetForRecall(Dataset):
    def __init__(
            self,
            text_all,
            tokenizer
    ):
        self.tokenizer = tokenizer
        self.text_all = text_all
        self.max_len = 1024

    def __len__(self):
        return len(self.text_all)

    def __getitem__(self, item):
        text = self.text_all[item]
        inputs = self.tokenizer(text,
                                return_tensors=None,
                                add_special_tokens=False,
                                max_length=self.max_len,
                                truncation=True)
        return inputs


class collater():
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.max_len = 1024
        self.pad_to_multiple_of = 8
        self.label_pad_token_id = -100
        warnings.filterwarnings("ignore",
                                message="`max_length` is ignored when `padding`=`True` and there is no truncation strategy.")

    def __call__(self, data):
        return self.tokenizer.pad(
            data,
            padding=True,
            max_length=self.max_len,
            pad_to_multiple_of=8,
            return_tensors='pt',
        )


def inference(df, model, tokenizer, device):
    batch_size = 12
    max_length = 512
    sentences = list(df['query_text'].values)
    pids = list(df['order_index'].values)
    all_embeddings = []
    length_sorted_idx = np.argsort([-len(sen) for sen in sentences])
    sentences_sorted = [sentences[idx] for idx in length_sorted_idx]
    # for start_index in trange(0, len(sentences), batch_size, desc="Batches", disable=False):

    dataset = DatasetForRecall(sentences_sorted, tokenizer) 
    sampler = DistributedSampler(dataset, shuffle=False, rank=dist.get_rank())  # rank 参数可省略 
    dataloader = DataLoader(dataset, batch_size=batch_size, sampler=sampler,
                             collate_fn=collater(tokenizer))

    for inputs in tqdm(dataloader):
        # print(inputs, device)
        inputs = inputs.to(device)
        with torch.no_grad():
            outputs = model.model(**inputs)
            embeddings = last_token_pool(outputs.last_hidden_state, inputs['attention_mask'])
            embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
            embeddings = embeddings.detach().cpu().numpy().tolist()
        all_embeddings.extend(embeddings)

    # end_pos = int(len(sentences)/batch_size) + 2
    # start_index = 0
    # for ii in tqdm(range(end_pos)):
    #     if start_index >= len(sentences):
    #         break
    #     sentences_batch = sentences_sorted[start_index: start_index + batch_size]
    #     features = tokenizer(sentences_batch, max_length=max_length, padding=True, truncation=True,
    #                          return_tensors="pt")
    #     features = batch_to_device(features, device)
    #     with torch.no_grad():
    #         outputs = model.model(**features)
    #         embeddings = last_token_pool(outputs.last_hidden_state, features['attention_mask'])
    #         embeddings = torch.nn.functional.normalize(embeddings, dim=-1)
    #         embeddings = embeddings.detach().cpu().numpy().tolist()
    #     all_embeddings.extend(embeddings)

    #     start_index += batch_size

    all_embeddings = [np.array(all_embeddings[idx]).reshape(1, -1) for idx in np.argsort(length_sorted_idx)]

    sentence_embeddings = np.concatenate(all_embeddings, axis=0)
    result = {pids[i]: em for i, em in enumerate(sentence_embeddings)}
    return result


path_prefix = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
# model_path = "/kaggle/input/sfr-embedding-mistral/SFR-Embedding-2_R"
# lora_path="/kaggle/input/v7-recall/epoch_19_model/adapter.bin"

model_path = BASE_MODEL
lora_path= BEST_WEIGHT

# device='cuda'
VALID = False

tokenizer = AutoTokenizer.from_pretrained(lora_path.replace("/adapter.bin",""))
bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )
model = AutoModel.from_pretrained(model_path, quantization_config=bnb_config,device_map='auto')
config = LoraConfig(
        r=64,
        lora_alpha=128,
        target_modules=[
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "gate_proj",
            "up_proj",
            "down_proj",
        ],
        bias="none",
        lora_dropout=0.05,  # Conventional
        task_type="CAUSAL_LM",
    )
model = get_peft_model(model, config)
d = torch.load(lora_path, map_location=model.device)
model.load_state_dict(d, strict=False)
model = model.eval()
# model = model.to(device)


import torch.nn as nn
if torch.cuda.device_count() > 1:
     model = DataParallel(model)  

# 因为在模型最后一层添了一个数据并行 layer，所以要加上下面这个命令，模型才能做 .generate() 
model = model.module

print(model)

# # 输出模型的参数名和参数值
# for name, param in model.named_parameters():
#     if "base_model.model.layers.12.input_layernorm.weight"  in name:
#         print(f"参数名: {name}")
#         print(f"参数值: {param}")

task_description = 'Given a math question and a misconcepte incorrect answer, please retrieve the most accurate reason for the misconception.'


if VALID:
    # tra = pd.read_parquet("/kaggle/input/v1-parquet/v1_val.parquet")
    tra = pd.read_csv("/kaggle/input/recall-qwen2-5-epoch15/train_new.csv")
    print(tra.shape)
    tra = tra[tra.train_flag == 0].reset_index(drop=True)
else:
    tra = pd.read_csv(f"{path_prefix}/test.csv")
    print(tra.shape)

print(tra)

misconception_mapping = pd.read_csv(f"{path_prefix}/misconception_mapping.csv")
if tra.shape[0]<10:
    misconception_mapping = misconception_mapping.sample(n=30,random_state=2023)

misconception_mapping.MisconceptionName = 'Here is misconcepte name : \n' + misconception_mapping.MisconceptionName

print(misconception_mapping)

if VALID:
    train_data = []
    for _,row in tra.iterrows():
        for c in ['A','B','C','D']:
            if str(row[f"Misconception{c}Id"])!="nan":
                # print(row[f"Misconception{c}Id"])
                real_answer_id = row['CorrectAnswer']
                real_text = row[f'Answer{real_answer_id}Text']
                query_text =f"###question###:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n###Correct Answer###:{real_text}\n###Misconcepte Incorrect answer###:{c}.{row[f'Answer{c}Text']}"
                row['query_text'] = get_detailed_instruct(task_description,query_text)
                row['answer_id'] = row[f"Misconception{c}Id"]
                train_data.append(copy.deepcopy(row))
    train_df = pd.DataFrame(train_data)
    train_df['order_index'] = list(range(len(train_df)))
else:
    train_data = []
    for _,row in tra.iterrows():
        for c in ['A','B','C','D']:
            if c ==row['CorrectAnswer']:
                continue
            if f'Answer{c}Text' not in row:
                continue
            real_answer_id = row['CorrectAnswer']
            real_text = row[f'Answer{real_answer_id}Text']
            query_text =f"###question###:{row['SubjectName']}-{row['ConstructName']}-{row['QuestionText']}\n###Correct Answer###:{real_text}\n###Misconcepte Incorrect answer###:{c}.{row[f'Answer{c}Text']}"
            row['query_text'] = get_detailed_instruct(task_description,query_text)
            row['answer_name'] = c
            train_data.append(copy.deepcopy(row))
    train_df = pd.DataFrame(train_data)
    train_df['order_index'] = list(range(len(train_df)))
    
print(train_df.shape)

train_embeddings = inference(train_df, model, tokenizer, device)

misconception_mapping['query_text'] = misconception_mapping['MisconceptionName']
misconception_mapping['order_index'] = misconception_mapping['MisconceptionId']
doc_embeddings = inference(misconception_mapping, model, tokenizer, device)

sentence_embeddings = np.concatenate([e.reshape(1, -1) for e in list(doc_embeddings.values())])
index_text_embeddings_index = {index: paper_id for index, paper_id in
                                         enumerate(list(doc_embeddings.keys()))}

with open('index_text_embeddings_index.pkl', 'wb') as fw:
    pkl.dump(index_text_embeddings_index, fw)

predicts_test = []
predicts_scores_all = []
for _, row in tqdm(train_df.iterrows()):
    query_id = row['order_index']
    query_em = train_embeddings[query_id].reshape(1, -1)
    
    cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    predicts_scores_all.append(cosine_similarity.tolist())
    
    # sort_index = np.argsort(-cosine_similarity)[:25]
    # pids = [index_text_embeddings_index[index] for index in sort_index]
    # predicts_test.append(pids)

train_df['predicts_scores_all'] = predicts_scores_all

train_df.to_csv(SAVE_FILE_NAME, index=False)

# if VALID:
#     train_df['recall_ids'] = predicts_test
#     print(mapk([[data] for data in train_df['answer_id'].values],train_df['recall_ids'].values))
# else:
#     train_df['MisconceptionId'] = [' '.join(map(str,c)) for c in predicts_test]
#     sub = []
#     for _,row in train_df.iterrows():
#         sub.append(
#             {
#                 "QuestionId_Answer":f"{row['QuestionId']}_{row['answer_name']}",
#                 "MisconceptionId":row['MisconceptionId']
#             }
#         )
#     submission_df = pd.DataFrame(sub)
#     submission_df.to_csv({SAVE_FILE_NAME}}, index=False)
#     print("Submission file created successfully!")


def check_hit(row):
    answer_id = row['answer_id']
    recall_ids = row['recall_ids']
    if answer_id in recall_ids:
        return True
    else:
        return False
        
def check_hit_pos(row):
    answer_id = row['answer_id']
    recall_ids = row['recall_ids']
    if answer_id in recall_ids:
        return recall_ids.index(answer_id)
    else:
        return -1

if VALID:
    train_df['recall_hit'] = train_df.apply(check_hit, axis=1)
    train_df['recall_hit_pos'] = train_df.apply(check_hit_pos, axis=1)
    print(train_df.recall_hit.sum())
    print(train_df.recall_hit.sum() / len(train_df))

    train_df.to_csv('val_result_epoch45_14b.csv', index=None)

Writing main.py


In [7]:
BASE_MODEL = '/kaggle/input/qwen2-5-14b-instruct'
BEST_WEIGHT = '/kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/adapter.bin'
SAVE_FILE_NAME = 'recall_14b_result.csv'
VALID = 0


!torchrun --standalone --nnodes=1 --nproc_per_node 1 main.py {BASE_MODEL} {BEST_WEIGHT} {SAVE_FILE_NAME} {VALID}

BASE_MODEL = /kaggle/input/qwen2-5-14b-instruct
BEST_WEIGHT = /kaggle/input/recall-model-qwen14b-11-13-all/epoch_82_model/adapter.bin
SAVE_FILE_NAME = recall_14b_result.csv
VALID = 0
local_rank: 0, world_size: 1
device: cuda:0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████████████| 8/8 [02:15<00:00, 16.88s/it]
/kaggle/working/main.py:281: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to

In [8]:
import pickle as pkl

with open('index_text_embeddings_index.pkl', 'rb') as fr:
    index_text_embeddings_index = pkl.load(fr)

index_text_embeddings_index

{0: 2256,
 1: 2186,
 2: 2336,
 3: 455,
 4: 2546,
 5: 2137,
 6: 2486,
 7: 2093,
 8: 2069,
 9: 301,
 10: 860,
 11: 448,
 12: 1302,
 13: 1991,
 14: 1573,
 15: 166,
 16: 2556,
 17: 947,
 18: 2319,
 19: 228,
 20: 835,
 21: 2236,
 22: 339,
 23: 2476,
 24: 830,
 25: 1284,
 26: 2435,
 27: 227,
 28: 5,
 29: 433}

In [9]:
import pandas as pd 
import ast

recall_14b_result = pd.read_csv('recall_14b_result.csv')
recall_14b_result.predicts_scores_all = recall_14b_result.predicts_scores_all.apply(ast.literal_eval)
recall_14b_result

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,query_text,answer_name,order_index,predicts_scores_all
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,B,0,"[0.17942195608934242, 0.15162270994602522, 0.0..."
1,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,C,1,"[0.1388295772597239, 0.12761588956777814, 0.04..."
2,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,D,2,"[0.15723997214934116, 0.07162952316433291, 0.0..."
3,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,A,3,"[0.29837058253031756, 0.12191076109796484, 0.0..."
4,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,B,4,"[0.15879827321758455, 0.10047224253331422, 0.0..."
5,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,C,5,"[0.2939493537162434, 0.1384900967771472, 0.068..."
6,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,A,6,"[0.07631620296727704, 0.20862495987508822, 0.0..."
7,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,C,7,"[0.062484230887910996, 0.2540800929443776, 0.0..."
8,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,D,8,"[0.09755888783995914, 0.20700205432440555, 0.1..."


In [10]:
# recall_14b_result.iloc[0].predicts_scores_all

In [11]:
BASE_MODEL = '/kaggle/input/qwen32b'
BEST_WEIGHT = '/kaggle/input/recall-model-qwen-32b-1117-all/epoch_79_model/epoch_79_model/adapter.bin'
SAVE_FILE_NAME = 'recall_32b_result.csv'
VALID = 0


!torchrun --standalone --nnodes=1 --nproc_per_node 1 main.py {BASE_MODEL} {BEST_WEIGHT} {SAVE_FILE_NAME} {VALID}

BASE_MODEL = /kaggle/input/qwen32b
BEST_WEIGHT = /kaggle/input/recall-model-qwen-32b-1117-all/epoch_79_model/epoch_79_model/adapter.bin
SAVE_FILE_NAME = recall_32b_result.csv
VALID = 0
local_rank: 0, world_size: 1
device: cuda:0
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|████████████████| 17/17 [05:14<00:00, 18.53s/it]
/kaggle/working/main.py:281: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed 

In [12]:
recall_32b_result = pd.read_csv('recall_32b_result.csv')
recall_32b_result.predicts_scores_all = recall_32b_result.predicts_scores_all.apply(ast.literal_eval)
recall_32b_result = recall_32b_result.rename(columns={'predicts_scores_all': 'predicts_scores_all_32b'})
recall_32b_result

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,query_text,answer_name,order_index,predicts_scores_all_32b
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,B,0,"[0.22162052298028811, 0.20810793963353547, 0.2..."
1,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,C,1,"[0.21227195050576597, 0.2450149114241924, 0.20..."
2,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,D,2,"[0.23669162899403773, 0.17393302760913798, 0.2..."
3,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,A,3,"[0.37025458204304584, 0.21005483113976453, 0.2..."
4,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,B,4,"[0.3076607850924624, 0.227050867322248, 0.2000..."
5,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,C,5,"[0.2990030326556905, 0.1725137614174912, 0.233..."
6,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,A,6,"[0.18056645063425947, 0.3593886194139202, 0.18..."
7,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,C,7,"[0.19964218755857388, 0.362059034294802, 0.165..."
8,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,D,8,"[0.22734368363090596, 0.27044266236871195, 0.2..."


In [13]:
# order_index	predicts_scores_all

recall_14b_result = recall_14b_result.rename(columns={'predicts_scores_all': 'predicts_scores_all_14b'})
recall_14b_result = recall_14b_result[['order_index', 'predicts_scores_all_14b']]


In [14]:
df_final  = pd.merge(recall_32b_result, recall_14b_result, on='order_index', how='left')

In [15]:
df_final

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,query_text,answer_name,order_index,predicts_scores_all_32b,predicts_scores_all_14b
0,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,B,0,"[0.22162052298028811, 0.20810793963353547, 0.2...","[0.17942195608934242, 0.15162270994602522, 0.0..."
1,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,C,1,"[0.21227195050576597, 0.2450149114241924, 0.20...","[0.1388295772597239, 0.12761588956777814, 0.04..."
2,1869,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,Instruct: Given a math question and a misconce...,D,2,"[0.23669162899403773, 0.17393302760913798, 0.2...","[0.15723997214934116, 0.07162952316433291, 0.0..."
3,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,A,3,"[0.37025458204304584, 0.21005483113976453, 0.2...","[0.29837058253031756, 0.12191076109796484, 0.0..."
4,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,B,4,"[0.3076607850924624, 0.227050867322248, 0.2000...","[0.15879827321758455, 0.10047224253331422, 0.0..."
5,1870,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,Instruct: Given a math question and a misconce...,C,5,"[0.2990030326556905, 0.1725137614174912, 0.233...","[0.2939493537162434, 0.1384900967771472, 0.068..."
6,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,A,6,"[0.18056645063425947, 0.3593886194139202, 0.18...","[0.07631620296727704, 0.20862495987508822, 0.0..."
7,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,C,7,"[0.19964218755857388, 0.362059034294802, 0.165...","[0.062484230887910996, 0.2540800929443776, 0.0..."
8,1871,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,Instruct: Given a math question and a misconce...,D,8,"[0.22734368363090596, 0.27044266236871195, 0.2...","[0.09755888783995914, 0.20700205432440555, 0.1..."


In [16]:
from tqdm import tqdm
import numpy as np

predicts_test = []
predicts_scores_all = []
for _, row in tqdm(df_final.iterrows()):
    query_id = row['order_index']

    predicts_scores_all_32b = np.array(row['predicts_scores_all_32b'])
    predicts_scores_all_14b = np.array(row['predicts_scores_all_14b'])

    cosine_similarity = predicts_scores_all_32b * 0.6 + predicts_scores_all_14b * 0.4
    
    # query_em = train_embeddings[query_id].reshape(1, -1)
    
    # cosine_similarity = np.dot(query_em, sentence_embeddings.T).flatten()
    # predicts_scores_all.append(cosine_similarity.tolist())
    
    sort_index = np.argsort(-cosine_similarity)[:25]
    pids = [index_text_embeddings_index[index] for index in sort_index]
    predicts_test.append(pids)


df_final['MisconceptionId'] = [' '.join(map(str,c)) for c in predicts_test]
sub = []
for _,row in df_final.iterrows():
    sub.append(
        {
            "QuestionId_Answer":f"{row['QuestionId']}_{row['answer_name']}",
            "MisconceptionId":row['MisconceptionId']
        }
    )
submission_df = pd.DataFrame(sub)
submission_df.to_csv("submission.csv", index=False)
print("Submission file created successfully!")

9it [00:00, 5675.65it/s]

Submission file created successfully!


In [17]:
submission_df

,QuestionId_Answer,MisconceptionId
0,1869_B,2556 2319 947 228 2476 433 860 448 455 301 166...
1,1869_C,2556 2476 2319 455 947 228 433 5 2093 2137 448...
2,1869_D,2556 2476 301 1284 1573 2256 860 448 947 433 2...
3,1870_A,2256 830 1991 835 2556 301 2236 947 339 2186 1...
4,1870_B,1991 339 2256 2556 947 2476 1284 830 166 2093 ...
5,1870_C,2256 830 1991 2556 2236 947 2137 835 2336 2486...
6,1871_A,1573 2186 2319 301 228 227 1302 2435 2556 1284...
7,1871_C,1573 2186 227 301 1302 2319 228 2069 1284 2435...
8,1871_D,2319 227 2236 2069 1573 2186 1302 301 2486 860...
